In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt

# 데이터 로드
data = pd.read_csv('../document/cleaned_data.csv')

# created_at 변환 및 time_diff 계산
data["created_at"] = pd.to_datetime(data["created_at"])
data["time_diff"] = data["created_at"].diff().dt.total_seconds().fillna(0)

# 독립 변수와 타겟 변수 설정
selected_columns = ["turn","speed","accuracy","course","heading",
"time_diff","wind_direct","wind_speed","surface_curr_drc","surface_curr_speed","air_temperature",
"water_temperature","air_pressure","humidity","salinity","status_0.0","status_1.0","status_2.0",
"status_3.0","status_5.0","status_6.0","status_7.0","status_8.0","status_9.0","status_10.0",
"status_11.0","status_12.0","status_15.0"]  # 독립 변수
target_columns = ["lat", "lon"]  # 타겟 변수

# RobustScaler를 사용한 독립변수 스케일링
scaler = RobustScaler()
data[selected_columns] = scaler.fit_transform(data[selected_columns])

# 랜덤 포레스트를 위한 학습 데이터 준비
X = data[selected_columns].values  # 독립 변수
y_lat = data["lat"].values  # 위도 예측
y_lon = data["lon"].values  # 경도 예측

# 랜덤 포레스트 모델 생성 및 학습
rf_lat = RandomForestRegressor(n_estimators=100, random_state=42)
rf_lon = RandomForestRegressor(n_estimators=100, random_state=42)

rf_lat.fit(X, y_lat)
rf_lon.fit(X, y_lon)

# Feature Importance 계산
feature_importance_lat = rf_lat.feature_importances_
feature_importance_lon = rf_lon.feature_importances_

# 평균 중요도 계산 (위도 & 경도 예측 중요도 평균)
feature_importance_avg = (feature_importance_lat + feature_importance_lon) / 2

# 중요도 순 정렬
feature_names = np.array(selected_columns)
sorted_idx = np.argsort(feature_importance_avg)[::-1]

# Feature Importance 시각화
plt.figure(figsize=(10, 6))
plt.barh(feature_names[sorted_idx], feature_importance_avg[sorted_idx], color="blue")
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Feature Importance (Random Forest)")
plt.gca().invert_yaxis()  # 중요도가 높은 순서로 정렬
plt.show()

# 중요도 순위 출력
feature_importance_df = pd.DataFrame({"Feature": feature_names[sorted_idx], 
                                      "Importance": feature_importance_avg[sorted_idx]})
print(feature_importance_df)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.